In [6]:
# -*- coding:utf-8-*-
# Edited by bighead 19-2-13

import numpy as np
import pandas as pd
import cv2
import random
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
##### get hog feature
def get_hot_features(trainset):
    features=[]
    hog = cv2.HOGDescriptor('../data/hog.xml')
    
    for img in trainset:
        img = np.reshape(img, (28,28))
        cv_img = img.astype(np.uint8)
        
        hog_feature = hog.compute(cv_img)
        features.append(hog_feature)
        
    features = np.array(features)
    features = np.reshape(features, (-1, 324))
    
    return features

In [8]:
##### knn predict process
def predict(testset, trainset, train_labels):
    predict=[]
    count=0
    
    for test_vec in testset:
        count += 1
        
        knn_list=[]
        max_index=-1
        max_dist=0
        
        ### first k points
        for i in range(k):
            label = train_labels[i]
            train_vec = trainset[i]
            
            dist = np.linalg.norm(train_vec-test_vec)
            knn_list.append((dist, label))
        
        ### points remain
        for i in range(k, len(train_labels)):
            label = train_labels[i]
            train_vec = trainset[i]
            
            dist = np.linalg.norm(train_vec-test_vec)
            
            ### find the furthest point on the k points
            if max_index < 0:
                for j in range(k):
                    if max_dist < knn_list[j][0]:
                        max_index = j
                        max_dist = knn_list[max_index][0]
            
            if dist < max_dist:
                knn_list[max_index] = (dist, label)
                max_index=-1
                max_dist=0
        
        class_total = k
        class_count = [0 for i in range(class_total)]
        for dist, label in knn_list:
            class_count[label]+=1
        
        mmax = max(class_count)
        
        for i in range(class_total):
            if mmax == class_count[i]:
                predict.append(i)
                break
        
    return np.array(predict)

In [9]:
### set k
k = 10

In [10]:
### main
print("Start read the data")
time_1 = time.time()

raw_data = pd.read_csv('../data/train.csv', header=0)
data = raw_data.values

imgs=data[:, 1:]
labels=data[:,0]

features=get_hot_features(imgs)

### train test set split
features_train, features_test, label_train, label_test = train_test_split(imgs, labels, test_size=0.2, random_state=233)

time_2 = time.time()
print("read data finish! cost: {} seconds".format(time_2-time_1))


print("start training")
print("knn do not need to train")
time_3 = time.time()
print("training cost: {} seconds".format(time_3-time_2))

print("start predicting")
test_predict = predict(testset=features_test, trainset=features_train, train_labels=label_train)
time_4 = time.time()
print("predicting cost: {} seconds".format(time_4-time_3))

score = accuracy_score(label_test, test_predict)
print("The accruacy scores is {}".format(score))

Start read the data
read data finish! cost: 6.83203887939 seconds
start training
knn do not need to train
training cost: 0.00037693977356 seconds
start predicting
predicting cost: 3310.0124681 seconds
The accruacy scores is 0.963333333333
